In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install functools
!pip install varname
!pip install statsmodels
!pip install sklearn
!pip install seaborn
!pip install xgboost
!pip install scikit-optimize
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for functools
  Running setup.py clean for functools
Failed to build functools
ERROR: Could not build wheels for functools, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in index

In [2]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import os
import sys
import math
from varname import nameof
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels import robust
from statsmodels.tsa.api import VAR
from sklearn.metrics import r2_score
import seaborn as sns

# import a Lasso module for time series analysis
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error

# import Random Forest module
from sklearn.ensemble import RandomForestRegressor

# import XGBoost module
from xgboost import XGBRegressor
from skopt import gp_minimize
from skopt.space import Real, Integer
from functools import partial

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Bidirectional, LSTM, Dropout, SimpleRNN
from tensorflow.keras import backend as K
from tensorflow.keras.backend import clear_session
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import warnings
import scipy.optimize as spop
from scipy.optimize import minimize
import multiprocessing as multi
from sklearn.linear_model import LinearRegression
import math

In [3]:
warnings.filterwarnings(action='ignore',category=RuntimeWarning)

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
file=pd.read_csv('/content/gdrive/MyDrive/bond_and_docs.csv')
#file=pd.read_csv('/content/gdrive/MyDrive/bond_and_dOCmeansbon.csv')

In [26]:
# Y1: Argentina, Y2: Brazil, Y3: Mexico, Y4: Venezuela
#S1 and S2: DOCs in means 1 and 2

DD=pd.concat([file['Y1'],file['Y2'],file['Y3'],file['Y4'],file['S1'],file['S2']], axis=1)
yy=DD.loc[:,["Y3","Y4"]]
yy=yy/100



S=DD.loc[:,"S1":"S2"]
#S0=DD.loc[0:268,"S1"]
#S0=pd.DataFrame(S0)
#A=DD.loc[:269,"Y1":"Y2"]
yy=yy.values
S=S.values
#S0=S0.values
#A=A.values
yy=np.mat(yy)
S=np.mat(S)
#S0=np.mat(S0)
#A=np.mat(A)
n=2
a=yy.shape
t=a[0]



In [27]:
## Fonction for constrained parameters

def trans4(c0):
    c1=c0

    c1[0]=c0[0]
    c1[1]=c0[1]
    c1[2]=c0[2]
    #c1[3]=c0[3]
    c1[3]=c0[3]
    c1[4]=c0[4]
    #c1[6]=c0[6]
    c1[5]=c0[5]

    c1[6]=math.exp(-c0[6])
    c1[7]=math.exp(-c0[7])
    c1[8]=math.exp(-c0[8])
    c1[9]=math.exp(-c0[9])


    return c1


## Density function

def v_prob(EV, HE):
    val= (1/math.sqrt(((2*math.pi)**n)*np.linalg.det(HE)))*math.exp(-0.5*EV.transpose().dot(np.linalg.inv(HE)).dot(EV))
    return val










In [39]:
## Log likelihood function
def lik_fcn4(prmtr1):
    global yyy, S


    yy1 = yyy
    S1 = S
    #S2=S0

    #A1=A

    prmtr=trans4(prmtr1)
    #Normal Times Parameters
    d1=prmtr[0]
    d2=prmtr[1]
    #d3=prmtr[2]
    d3=prmtr[2]
    d4=prmtr[3]
    #d6=prmtr[5]
    cst1=prmtr[4]
    cst2=prmtr[5]

    sigma_1=prmtr[6]
    sigma_2=prmtr[7]
    sigma1c=prmtr[8]
    sigma2c=prmtr[9]


    sigmat_1=np.mat([[sigma1c**2+sigma_1**2,sigma1c*sigma2c],[sigma1c*sigma2c,sigma2c**2+sigma_2**2]])



    likv=0
    j_iter=0
    while j_iter<t:



        e11=yy1[j_iter].transpose()-np.mat(cst1+ [S1[j_iter,0]*d1 + S1[j_iter,1]*d2 , cst2+ S1[j_iter,0]*d3 +S1[j_iter,1]*d4]).transpose()


        fy11=v_prob(e11,sigmat_1)


        fy=fy11

        likv=likv+np.log(fy)

        j_iter += 1



    return -likv



In [29]:
# Optimization values

params=[0.015,0.012, 0.014, 0.062,0.004,0.003, 0.021,0.039,0.015,0.014 ]


res=minimize(lik_fcn4,params,method="Nelder-Mead")
a=res.x

In [35]:
# Computation of the residual errors
u_1= yy[:,0]-a[4]-S[:,0]*a[0]-S[:,1]*a[1]
u_2=yy[:,1]-a[5]-S[:,0]*a[2]-S[:,1]*a[3]
z_1c=u_1/(2*a[8])
z_1=u_1/(2*a[6])
z_2c=u_2/(2*a[9])
z_2=u_2/(2*a[7])

z_1c=pd.DataFrame(z_1c)
z_1=pd.DataFrame(z_1)
z_2c=pd.DataFrame(z_2c)
z_2=pd.DataFrame(z_2)
Z=pd.concat([z_1c,z_1,z_2c,z_2], axis=1, join='inner' )
Z=Z.values
Z=np.mat(Z)
# Log likelihood value

Log_value=lik_fcn4(a)

# LR_0
Log_value_MS = 536.40 # Log likelihood for Markov switching is calculated once in GAUSS 13.
LR_0= 2*(Log_value - Log_value_MS)

In [44]:
# construction of i.i.d auxiliary random variables

SS=[]
BB=[]
CC=[]
for j in range (499):
  neta_1=np.random.normal(0,1,270)
  neta_2=np.random.normal(0,1,270)
  neta_c=np.random.normal(0,1,270)
  neta_1=pd.DataFrame(neta_1)
  neta_2=pd.DataFrame(neta_2)
  neta_c=pd.DataFrame(neta_c)
  neta_1=neta_1.values
  neta_2=neta_2.values
  neta_c=neta_c.values
  neta_1=np.mat(neta_1)
  neta_2=np.mat(neta_2)
  neta_c=np.mat(neta_c)
  neta_1=np.array(neta_1[:,0])
  neta_2=np.array(neta_2[:,0])
  neta_c=np.array(neta_c[:,0])
  SS.append(neta_1)
  BB.append(neta_2)
  CC.append(neta_c)

In [1]:
# Construction of 499 empirical LR stattistics

LRtest=[]
#params=[0.015,0.012,0.014,0.062,0.004,0.033,0.051,0.099,0.095,0.074]
params=[0.113,0.110,0.119,0.250,-0.212,0.231,0.210,-0.209,0.308,0.118]
#mu,sigma=0,1
for i in range (499):
  #neta_1=np.random.normal(mu,sigma,269)
  #neta_2=np.random.normal(mu,sigma,269)
  #neta_c=np.random.normal(mu,sigma,269)
  #neta_1=pd.DataFrame(neta_1)
  #neta_2=pd.DataFrame(neta_2)
  #neta_c=pd.DataFrame(neta_c)
  #neta_1=neta_1.values
  #neta_2=neta_2.values
  #neta_c=neta_c.values
  #neta_1=np.mat(neta_1)
  #neta_2=np.mat(neta_2)
  #neta_c=np.mat(neta_c)
  z1c=np.array(Z[:,0])
  z1=np.array(Z[:,1])
  z2c=np.array(Z[:,2])
  z2=np.array(Z[:,3])
  #neta_1=np.array(neta_1[:,0])
  #neta_2=np.array(neta_2[:,0])
  #neta_c=np.array(neta_c[:,0])
  #vec1=[]
  #vec2=[]

  #x=0
  #y=0
  #a_iter=0
  #while a_iter < 270:
    #x=res.x[6] + x*res.x[0] + S[a_iter,0]*res.x[1] + s[a_iter,0]*res.x[8]
    #y=res.x[7] +y*res.x[3] + S[a_iter,0]*res.x[4] +b[a_iter,0]*res.x[10]

    #vec1.append(x)
    #vec2.append(y)
    #a_iter += 1
  #F=res.x[6] + S[:,0]*res.x[1] + SS[i][:,0]*res.x[8]
  #G=res.x[7] + S[:,0]*res.x[4] + BB[i][:,0]*res.x[10]
  F=a[4] + S[:,0]*a[0] + S[:,1]*a[1]  + (z1c*CC[i])*a[8] + (z1*SS[i])*a[6]
  G=a[5] + S[:,0]*a[2] + S[:,1]*a[3] + (z2c*CC[i])*a[9] + (z2*BB[i])*a[7]
  Vec1=pd.DataFrame(F)
  Vec2=pd.DataFrame(G)
  Vec=pd.concat([Vec1,Vec2],axis=1,join='inner')
  yyy=Vec.values
  yyy=np.mat(yyy)
  res=minimize(lik_fcn4,params,method="Nelder-Mead")
  m1=lik_fcn4(res.x)
  m= 2*(m1 - Log_value_MS)

  LRtest.append(m)


LRtest_sorted=sorted(LRtest)

#LRtest_sorted=pd.DataFrame(LRtest_sorted)
#LRtest_sorted.replace([np.inf,-np.inf],0,inplace=True)
#bb=LRtest_sorted.values

Q=round(np.percentile(LRtest_sorted,95))
#Q=round(np.percentile(bb,95))


# Empirical p_value

DD=[]
for i in range (499):
   #if bb[i]>=LR_0:
  if LRtest_sorted[i]>=LR_0:
    a=1
    DD.append(a)

WB=(DD.count(1)+1)/400

# Compare the critical value with WB


NameError: ignored